In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.stats import ttest_ind
import random

from sklearn.preprocessing import LabelEncoder # for statistical analysis on gender

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report  # not sure if we keep this in the end

import joblib



%matplotlib inline

In [3]:
df = pd.read_csv('brain-proteomics.csv')

In [4]:
df.head()

,Case,years_to_birth,gender,histological_type,race,ethnicity,radiation_therapy,Grade,Mutation.Count,Percent.aneuploidy,...,p27_p,p27_pT157_p,p27_pT198_p,p38_pT180_Y182_p,p53_p,p62-LCK-ligand_p,p70S6K_p,p70S6K_pT389_p,p90RSK_p,p90RSK_pT359_S363_p
0,TCGA-CS-4938,31,female,astrocytoma,white,not hispanic or latino,no,G2,15,0.069412,...,-0.425127,-0.033398,0.289192,1.060163,-0.407456,-0.470354,-0.107559,-0.060441,-0.053104,-0.011132
1,TCGA-CS-6665,51,female,astrocytoma,white,not hispanic or latino,yes,G3,75,0.524814,...,0.076536,0.011809,-0.047973,-0.054275,-0.244020,0.106282,-0.034369,0.585072,0.431720,-0.201412
2,TCGA-CS-6666,22,male,astrocytoma,white,not hispanic or latino,yes,G3,18,0.403165,...,0.110268,0.066886,-0.066150,0.250434,0.432187,-0.210412,0.290949,-0.893383,-0.693677,-0.055250
3,TCGA-DB-5270,38,female,oligoastrocytoma,white,not hispanic or latino,no,G3,16,0.061382,...,-0.235321,0.015372,-0.127422,-1.190789,0.105396,0.218569,-0.099136,1.990618,0.166788,-0.235380
4,TCGA-DB-5273,33,male,astrocytoma,white,not hispanic or latino,yes,G3,16,0.017349,...,-0.343212,-0.250564,-0.234582,0.648598,-0.071851,-0.041811,-0.099300,-0.920359,-0.243159,-0.128841


In [52]:
gender_age_outcome = df[['years_to_birth', 'gender', 'outcome']]

gender_age_outcome.rename(columns={'years_to_birth': 'age'}, inplace=True)

## About the dataset

In [53]:
fig = px.histogram(gender_age_outcome, x="age",  color="gender", marginal="box",
                   hover_data=gender_age_outcome.columns)
fig.show()

In [47]:
outcome_mapping = {0: 'Oligodendroglioma', 1: 'Astrocytoma'}

In [48]:
# Replace numeric values in the 'outcome' column with their corresponding labels
gender_age_outcome['outcome'] = gender_age_outcome['outcome'].map(outcome_mapping)

In [49]:
# Calculate the percentages using value_counts(normalize=True)
outcome_percentages = gender_age_outcome['outcome'].value_counts(normalize=True) * 100


In [51]:
# Plot the pie chart with real names as labels
fig = px.pie(names=outcome_percentages.index, values=outcome_percentages.values,
             title='Distribution of the two different outcomes', labels=outcome_percentages.index)

# Show the plot
fig.show()

## Statistical Analysis

### Statistical analysis of the 176 proteins

In [10]:
proteins_outcome = df.drop(['Case', 'years_to_birth', 'gender', 'histological_type', 'race',
       'ethnicity', 'radiation_therapy', 'Grade', 'Mutation.Count',
       'Percent.aneuploidy', 'IDH.status'], axis=1)

In [11]:
from scipy.stats import ttest_ind

# Initialize an empty list to store protein names and corresponding p-values
protein_p_values = []

# Loop through each protein column and perform t-test
for column in proteins_outcome.drop('outcome', axis=1):  # Exclude the 'outcome' column
    group_0 = proteins_outcome[proteins_outcome['outcome'] == 0][column]
    group_1 = proteins_outcome[proteins_outcome['outcome'] == 1][column]
    t_stat, p_val = ttest_ind(group_0, group_1)
    protein_p_values.append({'Protein': column, 'p-value': p_val})

# Create a DataFrame from the list of dictionaries
p_values_df = pd.DataFrame(protein_p_values)

# Sort the DataFrame by p-values
p_values_df = p_values_df.sort_values(by='p-value')

# Reset index
p_values_df.reset_index(drop=True, inplace=True)

# Print the DataFrame
p_values_df

,Protein,p-value
0,Syk_p,3.385636e-11
1,YAP_pS127_p,1.678460e-09
2,AR_p,3.858150e-09
3,ACC1_p,7.671502e-07
4,YAP_p,1.139554e-06
...,...,...
169,PEA15_pS116_p,9.517016e-01
170,ASNS_p,9.659298e-01
171,Fibronectin_p,9.669654e-01
172,Cyclin_E2_p,9.938696e-01


In [12]:
# Bonferoni correction for p-values


# Number of comparisons (number of protein columns)
num_comparisons = len(proteins_outcome.drop('outcome', axis=1))

# Perform Bonferroni correction
p_values_df['Adjusted p-value'] = p_values_df['p-value'] * num_comparisons

# Ensure adjusted p-values do not exceed 1
p_values_df['Adjusted p-value'] = p_values_df['Adjusted p-value'].clip(upper=1)

# Print the DataFrame with adjusted p-values
p_values_df

,Protein,p-value,Adjusted p-value
0,Syk_p,3.385636e-11,1.036005e-08
1,YAP_pS127_p,1.678460e-09,5.136089e-07
2,AR_p,3.858150e-09,1.180594e-06
3,ACC1_p,7.671502e-07,2.347480e-04
4,YAP_p,1.139554e-06,3.487034e-04
...,...,...,...
169,PEA15_pS116_p,9.517016e-01,1.000000e+00
170,ASNS_p,9.659298e-01,1.000000e+00
171,Fibronectin_p,9.669654e-01,1.000000e+00
172,Cyclin_E2_p,9.938696e-01,1.000000e+00


In [13]:
p_values_df[p_values_df['Adjusted p-value']<= 0.05]

,Protein,p-value,Adjusted p-value
0,Syk_p,3.385636e-11,1.036005e-08
1,YAP_pS127_p,1.678460e-09,5.136089e-07
2,AR_p,3.858150e-09,1.180594e-06
3,ACC1_p,7.671502e-07,2.347480e-04
4,YAP_p,1.139554e-06,3.487034e-04
5,HER3_pY1289_p,1.153899e-06,3.530932e-04
6,c-Kit_p,2.440837e-06,7.468962e-04
7,ACC_pS79_p,4.694171e-06,1.436416e-03
8,STAT3_pY705_p,8.408725e-06,2.573070e-03
9,DJ-1_p,1.042737e-05,3.190774e-03


In [14]:
P17_list = p_values_df[p_values_df['Adjusted p-value']<= 0.05]['Protein'].tolist()
P17_list

['Syk_p',
 'YAP_pS127_p',
 'AR_p',
 'ACC1_p',
 'YAP_p',
 'HER3_pY1289_p',
 'c-Kit_p',
 'ACC_pS79_p',
 'STAT3_pY705_p',
 'DJ-1_p',
 '53BP1_p',
 'p27_p',
 'PDK1_p',
 'S6_pS235_S236_p',
 'PRDX1_p',
 'Bax_p',
 'IRS1_p']

### Visaulization of the 17 significant proteins

In [15]:
# Define the mapping from numeric values to labels
outcome_mapping = {0: 'Oligodendroglioma', 1: 'Astrocytoma'}

In [16]:
# Melt the dataframe to convert specific columns into a single column
melted_df_sign = df.melt(id_vars='outcome', value_vars=P17_list, var_name='Proteins', value_name='Protein levels')

# Replace numeric values in the 'outcome' column with their corresponding labels
melted_df_sign['outcome'] = melted_df_sign['outcome'].map(outcome_mapping)

# Plot the boxplot using Plotly
fig = px.box(melted_df_sign, x='Proteins', y='Protein levels', color='outcome',
             title='Boxplots of the differentially expressed proteins',
             category_orders={'outcome': ['Oligodendroglioma', 'Astrocytoma']})

# Show the plot
fig.show()

In [54]:
import random


# Select 5 random proteins from the P17_list
random_proteins = random.sample(P17_list, 5)

# Melt the dataframe to convert the selected columns into a single column
melted_df_random = df.melt(id_vars='outcome', value_vars=random_proteins, var_name='Proteins', value_name='Protein Levels')

# Replace numeric values in the 'outcome' column with their corresponding labels
melted_df_random['outcome'] = melted_df_random['outcome'].map(outcome_mapping)

# Plot the boxplot using Plotly
fig = px.box(melted_df_random, x='Proteins', y='Protein Levels', color='outcome',
             title='Boxplot of 5 Randomly Selected Proteins',
             category_orders={'outcome': ['Oligodendroglioma', 'Astrocytoma']})

# Show the plot
fig.show()

### Statistical analysis of the age and gender

In [18]:
gender_age_outcome = df[['years_to_birth', 'gender', 'outcome']]

In [19]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()

# Encode the 'gender' column
gender_age_outcome['gender_encoded'] = label_encoder.fit_transform(gender_age_outcome['gender'])

In [20]:
gender_age_outcome.head()

,years_to_birth,gender,outcome,gender_encoded
0,31,female,1,0
1,51,female,1,0
2,22,male,1,1
3,38,female,1,0
4,33,male,1,1


In [21]:
# Perform independent t-tests for 'age' and 'gender' columns based on the 'outcome' column
t_stat_age, p_val_age = ttest_ind(gender_age_outcome[gender_age_outcome['outcome'] == 0]['years_to_birth'], 
                                  gender_age_outcome[gender_age_outcome['outcome'] == 1]['years_to_birth'])
t_stat_gender, p_val_gender = ttest_ind(gender_age_outcome[gender_age_outcome['outcome'] == 0]['gender_encoded'], 
                                        gender_age_outcome[gender_age_outcome['outcome'] == 1]['gender_encoded'])

# Print the p-values for 'age' and 'gender' columns
print(f"p-value for age: {p_val_age}")
print(f"p-value for gender: {p_val_gender}")

p-value for age: 0.01106098915501576
p-value for gender: 0.7748621104407485


In [22]:
X = df.drop(['Case', 'gender', 'histological_type', 'race', 'ethnicity', 'radiation_therapy', 'Grade', 'Mutation.Count', 'Percent.aneuploidy', 'IDH.status', 'outcome'], axis = 1)

y = df['outcome']

In [23]:
P17_list = ['Syk_p',
 'YAP_pS127_p',
 'AR_p',
 'ACC1_p',
 'YAP_p',
 'HER3_pY1289_p',
 'c-Kit_p',
 'ACC_pS79_p',
 'STAT3_pY705_p',
 'DJ-1_p',
 '53BP1_p',
 'p27_p',
 'PDK1_p',
 'S6_pS235_S236_p',
 'PRDX1_p',
 'Bax_p',
 'IRS1_p']

In [24]:
X_P17 = X[['years_to_birth'] + P17_list]

In [25]:
from sklearn.model_selection import train_test_split

# train test split 

X_train, X_val_test, y_train, y_val_test = train_test_split(X_P17, y, test_size=0.35, stratify=y)


X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, stratify=y_val_test)

In [26]:
from imblearn.over_sampling import SMOTE

# doubling the datapoints for each class of the training set

sampler = SMOTE(sampling_strategy={0: y_train.value_counts()[0]*2, 1: y_train.value_counts()[1]*2})

In [27]:
# reassigning X_train - y_train after SMOTE

X_train, y_train = sampler.fit_resample(X_train, y_train)

In [28]:
from sklearn.preprocessing import MinMaxScaler


preprocessor = MinMaxScaler()


preprocessor

MinMaxScaler()

In [29]:
# fit and transform X_train on the preprocessor

X_train = preprocessor.fit_transform(X_train)

In [30]:
# Transform X_val and X_test

X_val = preprocessor.transform(X_val)

X_test = preprocessor.transform(X_test)

In [31]:
features_P17_age = X_P17.columns

## the model

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
log_reg_model_X = LogisticRegression(max_iter=1000)

log_reg_model_X.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [34]:
y_pred_log_X = log_reg_model_X.predict(X_val)

In [35]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred_log_X)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_val, y_pred_log_X))

Accuracy: 0.7592592592592593

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.52      0.63        21
           1       0.75      0.91      0.82        33

    accuracy                           0.76        54
   macro avg       0.77      0.72      0.73        54
weighted avg       0.76      0.76      0.75        54



In [36]:
import joblib


# Save the models

joblib.dump(log_reg_model_X, 'log_reg_model.pkl')

['log_reg_model.pkl']

In [37]:
# load the saved model 

loaded_log_reg_model = joblib.load('log_reg_model.pkl')

# Make predictions using the loaded model

log_reg_predictions = loaded_log_reg_model.predict(X_val)

# Evaluate the accuracy of the predictions
log_reg_accuracy = accuracy_score(y_val, log_reg_predictions)

print("Log Reg Model Accuracy:", log_reg_accuracy)

Log Reg Model Accuracy: 0.7592592592592593
